<h1><b>Will You Survive On Titanic AI</b></h1>
<h2>Importing Libraries and Database</h2>

In [1]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns

In [2]:
titanic = pd.read_csv(r"titanic.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
# sns.pairplot(titanic, hue="Survived")

<h1>Data Cleanup</h1>

In [4]:
titanic = titanic.drop(columns=['Name', 'Cabin','Embarked','Ticket','PassengerId'])
titanic = titanic[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]
titanic.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,3,male,34.5,0,0,7.8292,0
1,3,female,47.0,1,0,7.0000,1
2,2,male,62.0,0,0,9.6875,0
3,3,male,27.0,0,0,8.6625,0
4,3,female,22.0,1,1,12.2875,1


In [5]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row,column]-min1)/(max1-min1)
                x.at[row,column]=xprim 
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [6]:
DataProcessing.shuffle(titanic)

In [7]:


for row in range(len(titanic)-1,-1,-1):
    if titanic.at[row,'Sex']=="male":
        titanic.at[row,'Sex']=1
    elif titanic.at[row,'Sex']=="female":
        titanic.at[row,'Sex']=0
    else:
        titanic = titanic.drop(row,axis=0)

        
# values=titanic.select_dtypes(exclude="object")
# columnNames=values.columns.tolist()
# for column in columnNames:
# column = "PassengerId"
# for row in range(len(titanic)-1,-1,-1):
#     print(titanic.at[row,column])
#     temp = titanic.at[row,column]
#     if isinstance(temp, (int, float, complex)):
#         print(titanic.at[row,column])
        
        
#     else:
#         print(titanic.iloc[row])
#         titanic = titanic.drop(row,axis=0)


titanic.head()


,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,2,1,47.0,0,0,10.500,0
1,3,1,24.0,0,0,7.250,0
2,1,0,45.0,0,0,262.375,1
3,3,0,NaN,1,0,15.500,1
4,2,1,26.0,0,0,13.000,0


In [8]:
DataProcessing.normalization(titanic)
titanic.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,0.5,1,0.617566,0.000,0.0,0.020495,0
1,1.0,1,0.314256,0.000,0.0,0.014151,0
2,0.0,0,0.591191,0.000,0.0,0.512122,1
3,1.0,0,NaN,0.125,0.0,0.030254,1
4,0.5,1,0.340630,0.000,0.0,0.025374,0


In [9]:
titanic = titanic.dropna()

titanic.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,0.5,1,0.617566,0.0,0.000000,0.020495,0
1,1.0,1,0.314256,0.0,0.000000,0.014151,0
2,0.0,0,0.591191,0.0,0.000000,0.512122,1
4,0.5,1,0.340630,0.0,0.000000,0.025374,0
5,1.0,0,0.472504,0.0,0.222222,0.031035,1


In [10]:
trainingSet, validatingSet = DataProcessing.split(titanic,0.7)

In [20]:
titanic

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,1.0,0,0.301068,0.000,0.000000,0.015330,1
1,1.0,1,0.274693,0.000,0.000000,0.015412,0
2,1.0,1,0.406567,0.000,0.000000,0.015094,0
4,1.0,0,0.380193,0.000,0.000000,0.015469,1
5,0.5,1,0.498879,0.125,0.000000,0.040989,0
...,...,...,...,...,...,...,...
412,0.5,1,0.261506,0.000,0.000000,0.027058,0
413,0.0,1,0.657128,0.125,0.111111,0.412821,0
414,1.0,0,0.010946,0.125,0.111111,0.032596,1
415,0.5,1,0.551629,0.000,0.000000,0.025374,0


<h2>k-nearest neighbors algorithm</h2>

In [21]:
class Knn:
    @staticmethod
    def knn(x,element,k=1):
        dist = []
        y = 0
        for i in range(len(x)):
            sum = 0
            for j in range(len(element)-1):
                sum += (x.iloc[i,j] - element.iloc[j])**2
            dist.append(sum**(1/2))
        x.loc[:,"dist"]=dist
        y = x.sort_values("dist")
        kElementów = {}
        for i in range(k):
            kElementów[y["Survived"].iat[i]] = 0
        for i in range(k):
            kElementów[y["Survived"].iat[i]] +=1
        return max(kElementów, key=kElementów.get)
    @staticmethod
    def accuracy(x,z,k):
        good = 0
        for i in range(len(z)):
            if( Knn.knn(x,z.iloc[i],k) == z.iloc[i][6] ):
                good +=1
        return good/z.shape[0]

In [22]:
Knn.accuracy(trainingSet,validatingSet,3)

C:\Users\rafal\AppData\Local\Temp\ipykernel_1960\4069727225.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:,"dist"]=dist


1.0

In [23]:
#knn test
for i in range(5):
    DataProcessing.shuffle(titanic)
    print(Knn.accuracy(trainingSet, validatingSet,4))

C:\Users\rafal\AppData\Local\Temp\ipykernel_1960\4069727225.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:,"dist"]=dist


1.0
1.0
1.0
1.0
1.0


In [15]:
validatingSet.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
288,0.5,0,0.393380,0.000,0.000000,0.025374,1
289,0.0,1,0.591191,0.000,0.000000,0.057971,0
290,0.5,1,0.340630,0.000,0.000000,0.025374,0
291,0.5,0,0.314256,0.125,0.222222,0.126872,1
293,0.0,1,0.314256,0.125,0.000000,0.160574,0


<h2>Naive Bayes Algorithm</h2>

In [16]:
class NaiveBayes:
    @staticmethod
    def mean(column):
        return sum(column)/len(column)
    @staticmethod
    def stv(column, mean1):
        tmp=0
        for i in column:
            tmp+=(i-mean1)**2
        return np.sqrt(tmp/len(column))
    @staticmethod
    def gaussProbability(x, mu, sigma):
        if sigma == 0:
            return 0
        exponent=np.exp(-0.5*(x-mu)**2/(sigma**2))
        return 1/(sigma*np.sqrt(2*np.pi))*exponent
    @staticmethod
    def classify(x, sample):
        probability = []
        for className in x['Survived'].unique().tolist():
            p1 = 1
            columns = x.columns.tolist()[:-1]
            data = x[x['Survived'] == className]
            for column in columns:
                dataFromOneColumn = data[column]
                mu = NaiveBayes.mean(dataFromOneColumn)
                sigma = NaiveBayes.stv(dataFromOneColumn, mu)
                prob = NaiveBayes.gaussProbability(sample[column], mu, sigma)
                p1 *= prob
            p2 = x['Survived'].value_counts()[className]
            probability.append(p1*p2/len(x))
        maxIdx = probability.index(max(probability))
        return x['Survived'].unique().tolist()[maxIdx]
    @staticmethod
    def accuracy(trainSet, valSet):
        positiveCases = 0
        for idx, i in valSet.iterrows():
            if i['Survived'] == NaiveBayes.classify(trainSet, i):
                positiveCases += 1
        return positiveCases / len(valSet)

In [25]:
#naive test
arr = []
for i in range(20):
    DataProcessing.shuffle(titanic)
    arr.append(NaiveBayes.accuracy(trainingSet, validatingSet))
print(arr)
print(sum(arr)/len(arr))

[0.6161616161616161, 0.6464646464646465, 0.6060606060606061, 0.35353535353535354, 0.5555555555555556, 0.6262626262626263, 0.35353535353535354, 0.5757575757575758, 0.35353535353535354, 0.42424242424242425, 0.6161616161616161, 0.46464646464646464, 0.6767676767676768, 0.5555555555555556, 0.5454545454545454, 0.6060606060606061, 0.6666666666666666, 0.3838383838383838, 0.3333333333333333, 0.6363636363636364]
0.5297979797979797
